In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

IMG_SIZE = 64
CHANNELS = 1  


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
data_dir = '/content/drive/MyDrive/Autistic_cleandata'

def preprocess(img, label):
    img = tf.image.rgb_to_grayscale(img)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=(256, 256),  
    batch_size=32,
    label_mode='int'
)


Found 2897 files belonging to 2 classes.


In [ ]:
data = data.map(preprocess).cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)

train_size = int(0.75 * len(data))
val_size = int(0.15 * len(data))
test_size = int(0.1 * len(data))

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input

model = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE, CHANNELS)),
    Conv2D(8, (3,3), activation='relu'),
    MaxPooling2D(),
    Conv2D(16, (3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 8)      │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 16)     │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │        50,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,457 (201.00 KB)

 Trainable params: 51,457 (201.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2)

history = model.fit(train, epochs=20, validation_data=val, callbacks=[early_stop, reduce_lr])

Epoch 1/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 216s 69ms/step - accuracy: 0.5462 - loss: 0.6883 - val_accuracy: 0.6827 - val_loss: 0.6571 - learning_rate: 0.0010
Epoch 2/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 54s 81ms/step - accuracy: 0.6377 - loss: 0.6415 - val_accuracy: 0.6923 - val_loss: 0.5920 - learning_rate: 0.0010
Epoch 3/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.6805 - loss: 0.5954 - val_accuracy: 0.7163 - val_loss: 0.5735 - learning_rate: 0.0010
Epoch 4/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.6765 - loss: 0.5904 - val_accuracy: 0.6923 - val_loss: 0.5736 - learning_rate: 0.0010
Epoch 5/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - accuracy: 0.7106 - loss: 0.5640 - val_accuracy: 0.7067 - val_loss: 0.5716 - learning_rate: 0.0010
Epoch 6/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - accuracy: 0.7108 - loss: 0.5549 - val_accuracy: 0.7500 - val_loss: 0.5174 - learning_rate: 0.0010
Epoch 7/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - accuracy: 0.7125 - loss: 0.5331 - val_

In [ ]:
model.evaluate(test)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8801 - loss: 0.3234


[0.32774844765663147, 0.8680555820465088]

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
precision = Precision()
recall = Recall()
binary_accuracy = BinaryAccuracy()

for batch in test.as_numpy_iterator():
    X, y_true = batch
    y_pred = model.predict(X)

    precision.update_state(y_true, y_pred)
    recall.update_state(y_true, y_pred)
    binary_accuracy.update_state(y_true, y_pred)

print(f"Test Accuracy     : {binary_accuracy.result().numpy() * 100:.2f}%")
print(f"Test Precision    : {precision.result().numpy() * 100:.2f}%")
print(f"Test Recall       : {recall.result().numpy() * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Test Accuracy     : 81.25%
Test Precision    : 77.27%
Test Recall       : 90.67%


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

IMG_SIZE = 64  

img_path = '/content/drive/MyDrive/AutismDataset/test/Autistic.112.jpg'
img = cv2.imread(img_path)

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Input Image")
plt.show()

resize = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))

plt.imshow(resize.numpy().astype(np.uint8))
plt.axis('off')
plt.title("Resized Image")
plt.show()

input_tensor = np.expand_dims(resize / 255.0, axis=0)
if input_tensor.shape[-1] != 1:
    input_tensor = tf.image.rgb_to_grayscale(input_tensor)

pred = model.predict(input_tensor)

print(f"Raw prediction: {pred[0][0]:.4f}")
if pred > 0.5:
    print("Predicted: Not Autistic")
else:
    print("Predicted: Autistic")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Raw prediction: 0.6537
Predicted: Not Autistic


In [23]:
model.save('tiny_autism_model.h5')